El objetivo de este enfoque, sera generar un/os modelo/s para predecir los atributos del dia siguiente al ultimo disponible en el dataset. Aqui se aplicaran 2 enfoques:

- Un modelo que prediga todas las variablse en simultaneo (con el objetivo de captar la interrelacion entre las mismas).
- Un modelo que prediga solamente la variable target (incialmente se realizaran pruebas con la variable Close, y luego se procedera a usar la variable Tendencia).

Una vez realiza la prediccion de los atributos del dia siguiente, se procedera a realizar la prediccion de la Tendencia/Close, se realimientara el dataset, y se procedere a predecir otro dia, repitiendo esto N veces.

### Imports

In [166]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score
from pmdarima.arima import auto_arima
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from neuralprophet import NeuralProphet
from tensorflow.keras.layers import Reshape
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import datetime
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from sklearn.model_selection import GridSearchCV
from sklearn.base import RegressorMixin
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import make_scorer
from keras.callbacks import Callback
from sklearn.base import clone
from sklearn.ensemble import VotingRegressor

### Dataset

In [123]:
dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')

dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
dates = dataset['Open_time']

dataset.drop(['Sentimiento'], axis=1, inplace=True)
dataset.drop(['Sentimiento_coin'], axis=1, inplace=True)
dataset.drop(['Sentimiento_referentes'], axis=1, inplace=True)
dataset.drop(columns=['Open_time'], inplace=True)

dataset = dataset.round(2) # Limitar los valores float a 2 decimales en todo el dataframe

feature_dataset = dataset
feature_dataset.drop(['Tendencia'], axis=1, inplace=True)

validation = feature_dataset[-5:]
feature_dataset = feature_dataset[:-5]

### Scalers

In [124]:
scalers = {}
for col in feature_dataset.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    feature_dataset[col] = scaler.fit_transform(np.array(feature_dataset[col]).reshape(-1, 1))
    scalers[col] = scaler


In [125]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,...,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,0.501993,0.517208,0.492593,0.479569,0.290506,0.223416,0.168108,0.529131,0.532125,0.524609,...,0.133100,0.058824,0.00,0.00,0.166667,0.197248,0.536424,0.302018,0.212508,0.140182
1,0.479673,0.480653,0.464609,0.451664,0.253904,0.186206,0.134757,0.509543,0.502465,0.500765,...,0.077072,0.117647,0.25,0.25,0.000000,0.199541,0.576159,0.268600,0.191449,0.104869
2,0.451574,0.484737,0.463992,0.470999,0.265724,0.195353,0.140104,0.487853,0.498719,0.498122,...,0.131429,0.000000,0.00,0.00,0.000000,0.197248,0.423841,0.345523,0.296107,0.111289
3,0.470905,0.492514,0.484979,0.497708,0.209695,0.160164,0.126093,0.497502,0.528165,0.511054,...,0.134562,0.117647,0.25,0.00,0.166667,0.283257,0.509934,0.235183,0.276324,0.103799
4,0.497409,0.557457,0.513169,0.563883,0.348664,0.281482,0.277858,0.541868,0.610800,0.547961,...,0.132108,0.058824,0.25,0.00,0.000000,0.291284,0.470199,0.271753,0.403957,0.258962


(866, 64)


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,...,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
866,7.20,7.29,7.06,7.20,3362657.29,24151408.61,58901.0,47132.78,48170.00,46800.00,...,30.84,0.0,0.0,0.0,0.0,0.0,22.0,337.0,207.0,75000.0
867,7.20,7.34,7.08,7.10,4140070.61,29876047.34,74020.0,47751.08,48592.66,47557.16,...,46.13,0.0,0.0,0.0,0.0,0.0,12.0,303.0,272.0,86000.0
868,7.10,7.38,6.98,7.34,6638724.11,47756264.59,119442.0,48300.00,50334.82,47710.01,...,34.81,1.0,0.0,0.0,1.0,0.0,37.0,494.0,522.0,233000.0
869,7.34,7.45,7.11,7.35,5806068.65,42408110.28,106417.0,49917.28,50368.61,48300.95,...,30.40,0.0,0.0,0.0,0.0,0.0,54.0,373.0,354.0,95000.0
870,7.35,7.71,7.27,7.65,6998063.16,53082392.52,130998.0,49699.60,52043.71,49225.01,...,34.24,0.0,0.0,0.0,0.0,0.0,58.0,451.0,548.0,88000.0


(5, 64)

### Preparo el dataset para train: cada conjunto de entrenamiento, sera una seried de N dias previos, para predecir 1 dia siguiente.

In [126]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        end_ix = i + n_steps
        seq_x = data.iloc[i:end_ix, :].values
        seq_y = data.iloc[end_ix, :].values
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


n_steps = 30  # Longitud de la secuencia de entrada
n_features = feature_dataset.shape[1]  # Número de características

# Crear las secuencias de entrada y salida
X, y = create_sequences(feature_dataset, n_steps)

In [127]:
print(X[0].shape) # Cada dato de entrenamiento, es un conjunto de 30 dias con sus 64 features
print(y[0].shape) # El target de cada dato, son los 64 features del dia siguiente
print(X.shape)
print(y.shape)
print(feature_dataset.shape)

(30, 64)
(64,)
(836, 30, 64)
(836, 64)
(866, 64)


In [128]:
print(len(X))
print(len(y))

836
836


### Obtencion de los mejores hiperparametros

In [129]:
def custom_scoring(estimator, X, y):
    y_pred = estimator.predict(X)
    mse = mean_squared_error(y, y_pred)
    return -mse

In [130]:
# X_reshaped = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
# print(X_reshaped.shape)


In [146]:
import warnings
warnings.filterwarnings('ignore')

def create_model(activation, units, dropout, learning_rate):
    model = Sequential([
        LSTM(units=units, activation=activation, input_shape=(n_steps, n_features), return_sequences=True),
        Dropout(dropout),
        LSTM(units=units, activation=activation, return_sequences=True),
        Dropout(dropout),
        LSTM(units=units, activation=activation),
        Dropout(dropout),
        Dense(units=n_features)
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])
    return model

regressor = KerasRegressor(build_fn=create_model, verbose=0, activation='relu', units=50, dropout=0.2, learning_rate=0.1)

cv = TimeSeriesSplit(n_splits=10).split(X)
param_grid = {
    'activation': ['relu', 'tanh'],
    'units': [50, 100, 150],
    'dropout': [0.2, 0.3, 0.4],
    'learning_rate': [0.01, 0.001, 0.0001],
    'epochs': [10, 20, 30],
    'batch_size': [32, 64, 128]
}

grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, scoring=custom_scoring, cv=cv, verbose=10)
grid_result = grid_search.fit(X, y)

Fitting 10 folds for each of 486 candidates, totalling 4860 fits
[CV 1/10; 1/486] START activation=relu, batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01, units=50
[CV 1/10; 1/486] END activation=relu, batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01, units=50;, score=-0.022 total time=   5.9s
[CV 2/10; 1/486] START activation=relu, batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01, units=50
[CV 2/10; 1/486] END activation=relu, batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01, units=50;, score=-0.046 total time=   6.3s
[CV 3/10; 1/486] START activation=relu, batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01, units=50
[CV 3/10; 1/486] END activation=relu, batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01, units=50;, score=-0.035 total time=   7.0s
[CV 4/10; 1/486] START activation=relu, batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01, units=50
[CV 4/10; 1/486] END activation=relu, batch_size=32, dropout=0.2, epochs=10, learni

In [161]:
# Show best results
print("Best score:", grid_result.best_score_)
print("Best parameters:", grid_result.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = grid_result.best_estimator_
best_model.fit(X, y)

Best score: -0.011237594256515817
Best parameters: {'activation': 'tanh', 'batch_size': 32, 'dropout': 0.3, 'epochs': 20, 'learning_rate': 0.01, 'units': 50}


KerasRegressor(
	model=None
	build_fn=<function create_model at 0x7fcd11d5a8b0>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=20
	activation=tanh
	units=50
	dropout=0.3
	learning_rate=0.01
)

### Predicciones con el mejor conjunto de hiper parametros

In [163]:
n_days_to_predict = 5
future_dataset = feature_dataset

dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')
dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
dates = dataset['Open_time']

# Crear un DataFrame vacío para almacenar las predicciones desnormalizadas
predicted_values_desnormalized = pd.DataFrame(columns=future_dataset.columns)

# Lista para almacenar las fechas de las predicciones
predicted_dates = []

for _ in range(n_days_to_predict):
    # Predecir 1 día posterior al último día disponible en el dataset
    last_sequence = future_dataset.iloc[-n_steps:, :].values.reshape((1, n_steps, n_features))
    predictions = best_model.predict(last_sequence)

    # Agregar las predicciones sin desnormalizar a future_dataset
    predicted_values_normalized = pd.DataFrame(predictions, columns=future_dataset.columns)
    future_dataset = pd.concat([future_dataset, predicted_values_normalized], axis=0, ignore_index=True)

    # Desnormalizar las predicciones y agregarlas al DataFrame de predicciones desnormalizadas
    inverted_predictions = []
    for i in range(len(future_dataset.columns)):
        col = future_dataset.columns[i]
        scaler = scalers[col]
        prediction = predictions[:, i].reshape(-1, 1)
        inverted_prediction = scaler.inverse_transform(prediction)
        inverted_predictions.append(inverted_prediction)

    # Calcular la fecha del próximo día
    next_day_date = dates.iloc[-1] + pd.DateOffset(days=1)
    predicted_dates.append(next_day_date)

    # Actualizar la fecha del próximo día en el DataFrame principal
    dates = dates.append(pd.Series([next_day_date], name='Fecha'))

    # Crear un DataFrame con las predicciones desnormalizadas
    predicted_values_desnormalized = pd.concat([predicted_values_desnormalized,
                                                pd.DataFrame(np.concatenate(inverted_predictions, axis=1),
                                                             columns=future_dataset.columns)], 
                                                ignore_index=True)

# Agregar las fechas al DataFrame de predicciones desnormalizadas
predicted_values_desnormalized['Fecha'] = predicted_dates

print("Valores predichos para los próximos {} días:".format(n_days_to_predict))
display(future_dataset.tail(n_days_to_predict + 1))

print("Valores predichos desnormalizados para los próximos {} días:".format(n_days_to_predict))
display(predicted_values_desnormalized.tail(n_days_to_predict))


Valores predichos para los próximos 5 días:


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,...,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
865,0.066959,0.071359,0.069959,0.070759,0.167858,0.031327,0.037934,0.570251,0.605201,0.586566,...,0.160305,0.000000,0.000000,0.000000,0.000000,0.197248,0.271523,0.265448,0.241863,0.044409
866,0.059641,0.061797,0.058248,0.058260,0.188946,0.034690,0.056224,0.462954,0.457148,0.465500,...,0.145096,0.048976,-0.003844,0.053411,0.029628,0.219532,0.136571,0.190441,0.169860,0.030948
867,0.056221,0.058175,0.054974,0.055193,0.187182,0.033743,0.055097,0.450110,0.444251,0.451906,...,0.142941,0.051950,-0.004161,0.051587,0.032733,0.222244,0.135564,0.191697,0.171896,0.030742
868,0.046996,0.048871,0.046164,0.046352,0.181378,0.030004,0.051617,0.436072,0.430145,0.437588,...,0.140645,0.052381,-0.005918,0.048422,0.034011,0.223465,0.131811,0.187473,0.168229,0.028825
869,0.038121,0.039967,0.037566,0.037649,0.178403,0.026916,0.048993,0.425050,0.419150,0.426511,...,0.139668,0.052428,-0.007345,0.046805,0.034683,0.224013,0.128783,0.186425,0.167428,0.026653
870,0.028818,0.030612,0.028468,0.028445,0.175267,0.023745,0.046304,0.413990,0.408109,0.415492,...,0.138908,0.052125,-0.008634,0.045001,0.035027,0.224442,0.125830,0.184775,0.166002,0.024347


Valores predichos desnormalizados para los próximos 5 días:


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,...,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Fecha
0,6.642794,6.838205,6.390831,6.572905,7458796.0,52650488.0,157740.312500,39736.605469,40399.839844,39098.464844,...,0.832592,-0.015375,0.213645,0.177771,0.194321,20.622175,348.039459,297.170715,65841.640625,2024-02-15
1,6.471162,6.651932,6.231750,6.419021,7395661.0,51307164.0,154804.484375,39072.007812,39720.339844,38408.601562,...,0.883158,-0.016643,0.206347,0.196398,0.217964,20.470161,350.030975,300.361145,65457.617188,2024-02-16
2,6.008274,6.173443,5.803575,5.975478,7187876.0,46004144.0,145741.296875,38345.621094,38977.187500,37682.027344,...,0.890484,-0.023671,0.193688,0.204067,0.228617,19.903465,343.331573,294.614899,61874.003906,2024-02-17
3,5.562922,5.715510,5.385688,5.538855,7081398.5,41624152.0,138906.390625,37775.281250,38397.890625,37119.910156,...,0.891274,-0.029381,0.187222,0.208095,0.233393,19.446251,341.670105,293.358887,57814.632812,2024-02-18
4,5.096067,5.234400,4.943560,5.077105,6969117.0,37126088.0,131901.453125,37203.003906,37816.230469,36560.734375,...,0.886128,-0.034534,0.180004,0.210162,0.237138,19.000336,339.053772,291.124542,53503.851562,2024-02-19


### Guardado de los mejores hiperparametros

In [164]:
import json

# Obtener los hiperparámetros y puntajes de los 5 mejores modelos
top_n_models = 5
best_params_list = []
best_scores_list = []

for i in range(min(top_n_models, len(grid_search.cv_results_['params']))):
    best_params_list.append(grid_search.cv_results_['params'][i])
    best_scores_list.append(grid_search.cv_results_['mean_test_score'][i])

# Guardar los hiperparámetros de los 5 mejores modelos en un archivo JSON
with open('top_5_hyperparameters.json', 'w') as f:
    json.dump({'best_params': best_params_list, 'best_scores': best_scores_list}, f)

# O imprimir los hiperparámetros
print("Top 5 mejores modelos:")
for i in range(len(best_params_list)):
    print("Modelo", i+1)
    print("Hiperparámetros:", best_params_list[i])
    print("Puntaje:", best_scores_list[i])


Top 5 mejores modelos:
Modelo 1
Hiperparámetros: {'activation': 'relu', 'batch_size': 32, 'dropout': 0.2, 'epochs': 10, 'learning_rate': 0.01, 'units': 50}
Puntaje: -0.020468762380079275
Modelo 2
Hiperparámetros: {'activation': 'relu', 'batch_size': 32, 'dropout': 0.2, 'epochs': 10, 'learning_rate': 0.01, 'units': 100}
Puntaje: -0.031123289588577806
Modelo 3
Hiperparámetros: {'activation': 'relu', 'batch_size': 32, 'dropout': 0.2, 'epochs': 10, 'learning_rate': 0.01, 'units': 150}
Puntaje: -102.1820030484306
Modelo 4
Hiperparámetros: {'activation': 'relu', 'batch_size': 32, 'dropout': 0.2, 'epochs': 10, 'learning_rate': 0.001, 'units': 50}
Puntaje: -0.017034221045753285
Modelo 5
Hiperparámetros: {'activation': 'relu', 'batch_size': 32, 'dropout': 0.2, 'epochs': 10, 'learning_rate': 0.001, 'units': 100}
Puntaje: -0.01505311352736538


### Armado de un ensamble con los mejores 5 hiperparametros

In [173]:
## Clase personalizada para hacer el ensamble, dado que sklearn no provee ninguna clase que permita hacer ensmble
## de modelos re regresion multivariados
class MultivariableVotingRegressor:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)

    def predict(self, X):
        # Hacer predicciones con cada modelo
        predictions = [model.predict(X) for model in self.models]
    
        # Calcular el promedio de las predicciones
        average_predictions = np.mean(predictions, axis=0)
    
        return average_predictions


In [185]:

import json

# Leer los hiperparámetros desde el archivo JSON
with open('top_5_hyperparameters.json', 'r') as f:
    top_hyperparameters = json.load(f)

# Crear instancias de los modelos con los hiperparámetros
models = []
for params in top_hyperparameters['best_params']:
    model = KerasRegressor(build_fn=create_model, **params, random_state=42)
    model.fit(X, y)
    models.append(model)

# Crear un ensamble de modelos
ensemble = MultivariableVotingRegressor(models)

ensemble.fit(X, y)


Epoch 1/10
27/27 [==============================] - 5s 28ms/step - loss: 0.0490 - accuracy: 0.0514
Epoch 2/10
27/27 [==============================] - 1s 27ms/step - loss: 0.0320 - accuracy: 0.1041
Epoch 3/10
27/27 [==============================] - 1s 26ms/step - loss: 0.0263 - accuracy: 0.0825
Epoch 4/10
27/27 [==============================] - 1s 27ms/step - loss: 0.0171 - accuracy: 0.0849
Epoch 5/10
27/27 [==============================] - 1s 26ms/step - loss: 0.0152 - accuracy: 0.1053
Epoch 6/10
27/27 [==============================] - 1s 26ms/step - loss: 0.0145 - accuracy: 0.1017
Epoch 7/10
27/27 [==============================] - 1s 26ms/step - loss: 0.0143 - accuracy: 0.0993
Epoch 8/10
27/27 [==============================] - 1s 29ms/step - loss: 0.0139 - accuracy: 0.1017
Epoch 9/10
27/27 [==============================] - 1s 26ms/step - loss: 0.0138 - accuracy: 0.0909
Epoch 10/10
27/27 [==============================] - 1s 26ms/step - loss: 0.0133 - accuracy: 0.0981
Epoch 1/1

In [186]:
n_days_to_predict = 5
future_dataset = feature_dataset

dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')
dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])
dates = dataset['Open_time']

# Crear un DataFrame vacío para almacenar las predicciones desnormalizadas
predicted_values_desnormalized = pd.DataFrame(columns=future_dataset.columns)

# Lista para almacenar las fechas de las predicciones
predicted_dates = []

for _ in range(n_days_to_predict):
    # Predecir 1 día posterior al último día disponible en el dataset
    last_sequence = future_dataset.iloc[-n_steps:, :].values.reshape((1, n_steps, n_features))
    predictions = ensemble.predict(last_sequence)

    # Agregar las predicciones sin desnormalizar a future_dataset
    predicted_values_normalized = pd.DataFrame(predictions, columns=future_dataset.columns)
    future_dataset = pd.concat([future_dataset, predicted_values_normalized], axis=0, ignore_index=True)

    # Desnormalizar las predicciones y agregarlas al DataFrame de predicciones desnormalizadas
    inverted_predictions = []
    for i in range(len(future_dataset.columns)):
        col = future_dataset.columns[i]
        scaler = scalers[col]
        prediction = predictions[:, i].reshape(-1, 1)
        inverted_prediction = scaler.inverse_transform(prediction)
        inverted_predictions.append(inverted_prediction)

    # Calcular la fecha del próximo día
    next_day_date = dates.iloc[-1] + pd.DateOffset(days=1)
    predicted_dates.append(next_day_date)

    # Actualizar la fecha del próximo día en el DataFrame principal
    dates = dates.append(pd.Series([next_day_date], name='Fecha'))

    # Crear un DataFrame con las predicciones desnormalizadas
    predicted_values_desnormalized = pd.concat([predicted_values_desnormalized,
                                                pd.DataFrame(np.concatenate(inverted_predictions, axis=1),
                                                             columns=future_dataset.columns)], 
                                                ignore_index=True)

# Agregar las fechas al DataFrame de predicciones desnormalizadas
predicted_values_desnormalized['Fecha'] = predicted_dates

print("Valores predichos para los próximos {} días:".format(n_days_to_predict))
display(future_dataset.tail(n_days_to_predict + 1))

print("Valores predichos desnormalizados para los próximos {} días:".format(n_days_to_predict))
display(predicted_values_desnormalized.tail(n_days_to_predict))


1/1 [==============================] - 0s 30ms/step
Valores predichos para los próximos 5 días:


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,...,Compound total coin,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
865,0.066959,0.071359,0.069959,0.070759,0.167858,0.031327,0.037934,0.570251,0.605201,0.586566,...,0.160305,0.000000,0.000000,0.000000,0.000000,0.197248,0.271523,0.265448,0.241863,0.044409
866,0.070616,0.105223,0.080195,0.110610,0.113322,0.033917,0.067504,0.358003,0.340303,0.326819,...,0.098304,0.065903,0.008665,0.029302,0.041106,0.228132,0.097416,0.170511,0.143659,0.042319
867,0.070298,0.104832,0.079701,0.108852,0.112874,0.034250,0.066696,0.357076,0.338550,0.325861,...,0.097535,0.065080,0.008971,0.029248,0.041865,0.226349,0.097765,0.168763,0.143272,0.041660
868,0.069521,0.104490,0.078823,0.108243,0.112061,0.034111,0.066087,0.354724,0.336617,0.324025,...,0.096427,0.065116,0.009150,0.029208,0.041879,0.225148,0.097368,0.167374,0.142166,0.041336
869,0.068479,0.104065,0.077544,0.108082,0.110961,0.033694,0.065358,0.351203,0.334130,0.321264,...,0.095111,0.065584,0.009302,0.029292,0.041631,0.224153,0.096398,0.166080,0.140647,0.041103
870,0.067268,0.103439,0.075895,0.107642,0.109601,0.033216,0.064273,0.346698,0.330649,0.317580,...,0.093601,0.066045,0.009532,0.029520,0.041564,0.222938,0.095088,0.164521,0.138986,0.040686


Valores predichos desnormalizados para los próximos 5 días:


,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,...,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Fecha
0,7.193492,9.071614,7.457460,9.199312,4751622.0,51554068.0,187124.250000,34306.003906,34243.843750,32060.904297,...,1.120345,0.034659,0.117206,0.246638,0.269314,14.709836,316.429871,256.113373,87093.695312,2024-02-15
1,7.177553,9.051496,7.433469,9.111088,4735588.0,52026596.0,185017.890625,34258.007812,34151.527344,32012.292969,...,1.106366,0.035884,0.116990,0.251188,0.253761,14.762586,313.657501,255.506653,85862.101562,2024-02-16
2,7.138540,9.033930,7.390809,9.080544,4706497.5,51828712.0,183433.078125,34136.335938,34049.679688,31919.080078,...,1.106977,0.036602,0.116832,0.251274,0.243289,14.702525,311.454956,253.774475,85257.718750,2024-02-17
3,7.086262,9.012075,7.328616,9.072470,4667094.0,51237192.0,181533.703125,33954.128906,33918.632812,31779.009766,...,1.114933,0.037209,0.117169,0.249788,0.234614,14.556125,309.402924,251.393600,84822.382812,2024-02-18
4,7.025496,8.979892,7.248485,9.050423,4618441.5,50560060.0,178707.750000,33720.992188,33735.230469,31592.046875,...,1.122758,0.038127,0.118078,0.249386,0.224023,14.358318,306.930145,248.791107,84042.820312,2024-02-19
